# Import Modules

In [36]:
# import modules
import pandas as pd
import numpy as np
import spotipy

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:_.2f}'.format)
# NOTE: underscore separaters ('_') are better than commas (',') because 
# numbers with underscores work in Python without any extra effort.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

# Import Starting Data

### The Billboard 100
https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs

In [ ]:
# Billboard Top 100 Historical Data
# via:  https://toolbox.google.com/datasetsearch
url_billboard = r'D:\RYERSON\820\Datasets\Billboard The Hot 100 Songs\charts.csv'

df_billboard = pd.read_csv(url_billboard)
df_billboard['date'] = pd.to_datetime(df_billboard['date'])

# Unique Songs from The Billboard 100 Dataset

# just the songs on the billboard 100, once per song
df_billboard_songs = df_billboard[['song', 'artist']].drop_duplicates().sort_values(['artist', 'song']).reset_index(drop=True)

# add a blank id column and a blank MISSING column 
df_billboard_songs['id'] = ''
df_billboard_songs['MISSING'] = ''

df_billboard.shape, df_billboard_songs.shape

### 1.2M Songs with Metadata (csv)
https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs

In [ ]:
# Spotify 1.2M+ Songs
# via:  https://www.kaggle.com/datasets/
url_1M_songs = r'D:\RYERSON\820\Datasets\Spotify 1.2M+ Songs\tracks_features.csv'

# create the dataframe with the large number of songs metadata
df_1M_songs = pd.read_csv(url_1M_songs)

# make a list of song ids from the 1M dataset
metadata_ids_csv = df_1M_songs.id.to_list()

### 8.7M Songs with Metadata (SQL)

In [ ]:
# all ids from the SQLite database
metadata_ids_SQL = pd.read_csv('all_ids_sql.csv', header=None, names=['id'])
metadata_ids_SQL = metadata_ids_SQL.id.to_list()

# audio feature data not imported yet (very large)

### Combine Ids For Datasets with Metadata

In [ ]:
# list of ids for all of our known metadata
all_metadata_ids = set(metadata_ids_csv + metadata_ids_SQL) # set() faster to search, and no duplicates
len(all_metadata_ids)

### Get Track IDs using API

### TEMPORARY TOKEN WORKFLOW

##### get a temporary authorization token from: https://developer.spotify.com/console/get-search-item

In [ ]:
# input the temporary token
TEMP_TOKEN = input('Enter token: ')

# create a spotify object
spotify = spotipy.Spotify(auth=TEMP_TOKEN)

In [ ]:
# helper function
def find_id(track_title, artist_name, metadata_ids):
    """ 
    for searches with multiple results, all id were identical for the test cases I ran 
    some searches return no results, in this case the song is not on spotify
        confirmed by spot checks in the spotify music player
    some tracks give a 404 error, 
        these seem to exist in Spotify but 404 anyway
        not sure why the API does this but 
    """
    track_info = spotify.search(q='artist:' + artist_name + ' track:' + track_title, type='track')
    
    if track_info['tracks']['items'] == []:  # if track doesn't exist on Spotify
        return '', 'MISSING'
    else:
        # default to 0th id
        track_id = track_info['tracks']['items'][0]['id']
        
        number_of_results = len(track_info['tracks']['items'])
        
        # check if there is a better match
        for i in range(number_of_results):
            current_id = track_info['tracks']['items'][i]['id']
            if current_id in metadata_ids:
                return current_id, 'matched'  # immediately return it if it's found
        
        # if we made it through the loop without returning, note 'MISSING' and return the 0th id
        return track_id, 'MISSING'

In [ ]:
%%time
# TEST
find_id("You Can't Turn Me Off (In The Middle Of Turning Me On)", 'High Inergy', all_metadata_ids)

In [ ]:
# TEST
find_id("You Can't Find this (SONG)", 'Low Unergy', all_metadata_ids)

### Add Spotify IDs to billboard songs matched in the datasets

In [ ]:
# load saved csv if required
df_billboard_songs = pd.read_csv('df_billboard_songs.csv', keep_default_na=False)
df_billboard_songs.id.nunique()

In [ ]:
# start over at
start_over_at = 30000

# populate df_billboard_songs with ids, where available
for i, row in df_billboard_songs.iterrows():
    
    # start over at
    if i < start_over_at-1:
        continue
    
    # show status update
    if i%10 == 0:
        print(i, end='  ')
    if i%100 == 0:
        print()
        
    # start over where we finished (don't overwrite known ids)
    if df_billboard_songs['id'].iloc[i] != '':  
        continue
    # append id, NONE, or 'ERROR'
    else:
        artist = row[1]
        song = row[0]
        try:
            # unless there is an error, will append the id or None 
            df_billboard_songs['id'].iloc[i], df_billboard_songs['MISSING'].iloc[i] = find_id(song, artist, all_metadata_ids)
        except:  # all errors treated the same
            # if there is an error, change id to 'ERROR'
            print('ERROR:  ', artist, song)
            df_billboard_songs['MISSING'].iloc[i] = 'ERROR'  # leave id blank
        
        # save every 1000 rows, if new
        if i%1000 == 0:
            df_billboard_songs.to_csv('df_billboard_songs_TEMP.csv', index=False)
        
# save final dataframe
df_billboard_songs.to_csv('df_billboard_songs.csv', index=False)

### After Gathering all IDs

In [ ]:
# reload df_billboard_songs if required
df_billboard_songs = pd.read_csv('df_billboard_songs.csv', keep_default_na=False)

In [ ]:
# how many id have we added
df_billboard_songs.id.nunique(), sum(df_billboard_songs.id != "")
# 83 duplicated ids

### Remove Duplicates / Errors

In [ ]:
duplicates = df_billboard_songs[df_billboard_songs.id!='']
duplicates = duplicates[duplicates.id.duplicated(False)]
duplicates.to_csv('duplicated_ids.csv', index=True)

In [ ]:
# # go through this manually and fix it
#     # in this case NEED TO FIX BOTH BILLBOARD DATAFRAMES

# # OR drop the corrupted rows
# # manually imputing is problematic, error prone, and time consuming

# duplicates.sort_values('id').head(200)

In [ ]:
# set of duplicated ids
duplicated_ids = set(duplicates.id)

In [ ]:
sum(df_billboard_songs['id'].isin(duplicated_ids))

In [ ]:
# drop from billboard list of known ids 
for i, row in df_billboard_songs.iterrows():
    if df_billboard_songs.iloc[i]['id'] in duplicated_ids:
        df_billboard_songs['MISSING'].iloc[i] = 'DUPLICATED'
        df_billboard_songs['id'].iloc[i] = ''

In [ ]:
sum(df_billboard_songs['id'].isin(duplicated_ids))

In [ ]:
df_billboard_songs.to_csv('df_billboard_songs - duplicates removed.csv', index=False)

### what songs are still missing audio feature data?

In [ ]:
# reload the dataframe if required
df_billboard_songs = pd.read_csv('df_billboard_songs - duplicates removed.csv', keep_default_na=False)

In [ ]:
# how many id have we added
df_billboard_songs.id.nunique(), sum(df_billboard_songs.id != '')
# off by one because '' counts as a unique id

In [ ]:
# check how many id match the metadata_ids
sum(df_billboard_songs.id.isin(all_metadata_ids))

In [ ]:
# songs on spotify that we don't have audio features for yet
need_audio_features = df_billboard_songs[(~df_billboard_songs.id.isin(all_metadata_ids) & (df_billboard_songs.id != ''))]
need_audio_features.to_csv('need_audio_features.csv', index=False)

In [ ]:
len(need_audio_features.id)

### Use API again to get missing audio features
https://developer.spotify.com/console/get-audio-features-track/

In [ ]:
# input the temporary token
TEMP_TOKEN = input('Enter token: ')

# create a spotify object
spotify = spotipy.Spotify(auth=TEMP_TOKEN)

In [ ]:
# initialise dataframe

with_audio_features = need_audio_features.copy().reset_index()

list_of_features = [
    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'
]

for feature in list_of_features:
    with_audio_features[feature] = ''

with_audio_features.head()

In [ ]:
start_over_at = 1200

list_of_features = [
    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'
]

for i, row in with_audio_features.iterrows():
    
    # start over at
    if i < start_over_at-1:
        continue
    
    # show status update
    if i%10 == 0:
        print(i, end='  ')
    if i%100 == 0:
        print()
    
    track_id = with_audio_features['id'].iloc[i]
    temp_audio_features = spotify.audio_features(track_id)
    
    for key in list_of_features:
        with_audio_features[key].iloc[i] = temp_audio_features[0][key]
    
    if i%100 == 0:
        with_audio_features.to_csv('audio_features_TEMP.csv', index=True)
    
# save final df
with_audio_features.to_csv('audio_features_FINAL.csv', index=True)

### OPTIONAL QA SPOTCHECKS

In [ ]:
#### TODO: QA check ####
# should I confirm that i get the same audio features as from the other datasets??
# maybe do a small spotcheck

QA_DATAFRAME = df_billboard_songs[df_billboard_songs.MISSING == 'matched'].sample(100).reset_index()

list_of_features = [
    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'
]

for feature in list_of_features:
    QA_DATAFRAME[feature] = ''

QA_DATAFRAME.head()

In [ ]:
start_over_at = 0

list_of_features = [
    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'
]

for i, row in QA_DATAFRAME.iterrows():
    
    # start over at
    if i < start_over_at-1:
        continue
    
    # show status update
    if i%10 == 0:
        print(i, end='  ')
    if i%100 == 0:
        print()
    
    track_id = QA_DATAFRAME['id'].iloc[i]
    temp_audio_features = spotify.audio_features(track_id)
    
    for key in list_of_features:
        QA_DATAFRAME[key].iloc[i] = temp_audio_features[0][key]
    
# save final df
QA_DATAFRAME.to_csv('QA_DATAFRAME.csv', index=True)

In [ ]:
# check these later to confirm that the downloaded data is correct

# Finalize Dataset

### Reimport data from Billboard 100 and 1.2M Songs Dataset

In [15]:
# billboard songs 
df_billboard_songs = pd.read_csv('df_billboard_songs - duplicates removed.csv', keep_default_na=False)

# billboard songs with audio features from API (missing from other datasets)
df_api_features = pd.read_csv('audio_features_FINAL.csv', keep_default_na=False)
df_api_features = df_api_features[[
    'id', 'song', 'artist',
    'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 
    'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence'
]]

# billboard time series
url_billboard = r'D:\RYERSON\820\Datasets\Billboard The Hot 100 Songs\charts.csv'
df_billboard = pd.read_csv(url_billboard)
df_billboard['date'] = pd.to_datetime(df_billboard['date'])

# 1.2M songs with metadata
url_1M_songs = r'D:\RYERSON\820\Datasets\Spotify 1.2M+ Songs\tracks_features.csv'
df_1M_songs = pd.read_csv(url_1M_songs)
df_1M_songs.rename(columns={'name': 'song', 'artists': 'artist', 'duration': 'duration_ms'}, inplace=True)
df_1M_songs = df_1M_songs[[
    'id', 'song', 'artist',
    'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 
    'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence'
]]

### 8+ M. Spotify Tracks, Genre, Audio Features (SQLite)
https://www.kaggle.com/datasets/maltegrosse/8-m-spotify-tracks-genre-audio-features

In [26]:
url_8M_sql = 'D:\RYERSON\820\Datasets\8+ M. Spotify Tracks, Genre, Audio Features\spotify.sqlite'
url_8M_csv = 'all_audio_features_sql.csv'  # .gitignore (very big)

df_8M_songs = pd.read_csv(url_8M_csv, on_bad_lines='skip')
df_8M_songs.rename(columns={'name': 'artist', 'name:1': 'song', 'duration': 'duration_ms'}, inplace=True)
# already in alphabetical order

df_8M_songs.drop_duplicates(inplace=True)

### Merge Billboard 100 with Metadata

In [48]:
df_8M_songs.head(1)

,id,artist,song,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,1dizvxctg9dHEyaYTFufVi,Nancy Fletcher,Gz And Hustlas (feat. Nancy Fletcher),0.16,0.65,275893,0.81,0.00,1,0.36,-4.90,1,0.31,91.89,4,0.79


In [49]:
df_1M_songs.head(1)

,id,song,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,['Rage Against The Machine'],0.47,0.98,7,-5.40,1,0.07,0.03,0.00,0.36,0.50,117.91,210133,4.00


In [63]:
df_api_features.shape

(3573, 16)

In [62]:
df_billboard_songs.shape

(29681, 4)

In [ ]:
# STEP 1: Merge audio features with Billboard 100 Songs



# initialize new dataframe
all_audio_features = pd.DataFrame(
    data=np.nan,
    index=[x for x in range(29681)],
    columns=['id', 'song', 'artist']+list_of_features
)

# create sets of ids for fast searching
set_api_idx = set(df_api_features.id)
set_8M_idx = set(df_8M_songs.id)
set_1M_idx = set(df_1M_songs.id)

In [132]:
# STEP 1: Merge audio features with Billboard 100 Songs

# audio features
list_of_features = [
    'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 
    'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence'
]

# songs in the billboard 100 list
set_billboard = set(df_billboard_songs.id)
set_billboard.discard('')

# temporary dataframes
df1 = df_8M_songs[df_8M_songs.id.isin(set_billboard)]
df1 = df1[[
    'id', 'song', 'artist',
    'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 
    'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence'
]]
set1 = set_billboard - set(df1.id)

df2 = df_1M_songs[df_1M_songs.id.isin(set1)]
df2 = df2[[
    'id', 'song', 'artist',
    'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 
    'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence'
]]
set2 = set1 - set(df2.id)

df3a = df_billboard_songs[df_billboard_songs.id.isin(set2)].reset_index(drop=True)
df3b = pd.DataFrame(data=np.nan, index=[x for x in range(df3a.shape[0])], columns=list_of_features)
df3 = pd.concat([df3a, df3b], axis=1)
df3 = df3[[
    'id', 'song', 'artist',
    'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 
    'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence'
]]


# all_audio_features = pd.concat([df_api_features, df1, df2]).reset_index(drop=True)

In [159]:
df2.head(0)

,id,song,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [157]:

df3.head()


,id,song,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,11Aldbvo6UCcVhBzv4oUdw,What Now My Love,"""Groove"" Holmes",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5smPCwVq1pp9b9mnhKRxQj,Music Of My Heart,'N Sync & Gloria Estefan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14gC7IS8BFlzabcoqR1iNW,El Trago (The Drink),2 In A Room,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6byQuxXOHoos4pB8AnzuM3,Walk Tall,2 Of Clubs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2P7L26IbmkrqxxtyP1CRRb,Get Ready For This,2 Unlimited,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
df3[[
    'id', 'song', 'artist',
    'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 
    'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence'
]]

,id,song,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,11Aldbvo6UCcVhBzv4oUdw,What Now My Love,"""Groove"" Holmes",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5smPCwVq1pp9b9mnhKRxQj,Music Of My Heart,'N Sync & Gloria Estefan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14gC7IS8BFlzabcoqR1iNW,El Trago (The Drink),2 In A Room,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6byQuxXOHoos4pB8AnzuM3,Walk Tall,2 Of Clubs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2P7L26IbmkrqxxtyP1CRRb,Get Ready For This,2 Unlimited,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3568,6jRq8gaZhMUyy5RclRPcAf,Hot Girl Bummer,blackbear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3569,3anp5nvmUKwMBEIEL1sPHR,Lean & Dabb,iLoveMemphis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3570,2nG54Y4a3sH9YpfxMolOyi,Shy Away,twenty one pilots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3571,2q2pZHWEQe6w9PgACCzMtC,I Got It From My Mama,will.i.am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
sum(test.id.duplicated()), test.shape

(4148, (26337, 16))

In [115]:
test.sample(10)

,id,song,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
24047,1bHoWgbCixwEqq817VtNQa,Even It Up,['Heart'],0.70,0.75,9,-11.26,1,0.04,0.17,0.35,0.06,0.96,135.97,309893,4.00
22021,0dr71rw5oIcPtUHRJES6gP,Stand Up (Kick Love Into Motion),Def Leppard,0.60,0.78,4,-7.48,1,0.03,0.09,0.00,0.06,0.79,103.03,272871,4.00
19862,4h6xr0rwyNyLFl7Peeku9A,Together (Boys Get Hurt Remix),Re-Flex,0.69,0.83,8,-7.09,1,0.06,0.02,0.01,0.08,0.22,113.99,440720,4.00
8948,7zo8XAMYBG6nGpqGiIudBc,Angelito,Don Omar,0.63,0.66,1,-5.11,0,0.05,0.20,0.00,0.17,0.31,156.01,284933,4.00
25447,2xTzFWaVvNzmz1JXbhOt64,Through the Storm (with Elton John),"['Aretha Franklin', 'Elton John']",0.66,0.80,2,-10.10,1,0.07,0.28,0.00,0.34,0.59,99.56,261254,4.00
15282,3R8GUFHDWN3UzSzU7htCHB,Uh! Oh! Part 2,The Nutty Squirrels,0.71,0.42,4,-10.22,1,0.04,0.70,0.00,0.20,0.60,112.56,144693,4.00
19270,3PT25AfBT85mAQxqwyaLSh,Ridin' Low (feat. Darvy Traylor) - Radio Mix,L.a.d.,0.70,0.72,5,-6.53,1,0.07,0.16,0.00,0.05,0.67,197.56,252360,4.00
1358,68Dni7IE4VyPkTOH9mRWHr,No Role Modelz,J. Cole,0.69,0.52,10,-8.49,0,0.34,0.32,0.00,0.05,0.49,100.03,292799,4.00
22362,0MmluR9xqG0a15ecfJP2qy,The Return Of The Red Baron,The Royal Guardsmen,0.65,0.78,10,-9.74,1,0.13,0.64,0.00,0.17,0.82,134.62,165240,4.00
7818,57q8tP6DIvApDldr9Hvhnu,Baby Do the Philly Dog,The Olympics,0.62,0.56,1,-8.84,1,0.03,0.11,0.00,0.06,0.86,74.92,138344,4.00


In [99]:
# SLOWWWWWW

# loop through df_billboard_songs and populate all_audio_features billboard songs and audio features
for i, row in df_billboard_songs.iterrows():
    
    # show status update
    if i%10 == 0:
        print(i, end=' ')
    if i%100 == 0:
        print()
        
    # save the dataframe every 100 rows (before continue, just in case)
    if i%100 == 0:
        all_audio_features.to_csv('all_audio_features_TEMP.csv', index=False)
    
    # append id, song, and artist to all_audio_features
    idx, song, artist = row['id'], row['song'], row['artist']
    all_audio_features['id'].iloc[i] = idx
    all_audio_features['song'].iloc[i] = song
    all_audio_features['artist'].iloc[i] = artist
    
    # first check df_api_features
    if idx in set_api_idx:
        row_id = df_api_features[df_api_features.id == idx]
        
    # then check df_8M_songs
    elif idx in set_8M_idx:
        row_id = df_8M_songs[df_8M_songs.id == idx]
    
    # last check df_1M_songs
    elif idx in set_1M_idx:
        row_id = df_1M_songs[df_1M_songs.id == idx]
    
    # don't add features if none found
    else:
        continue
    
    # loop through and populate features (if found)
    for feature in list_of_features:
        all_audio_features[feature].iloc[i] = row_id[feature]
    
    
# save the dataframe
all_audio_features.to_csv('all_audio_features.csv', index=False)

0 
10 20 30 40 50 60 70 

ValueError: setting an array element with a sequence.

In [ ]:
all_audio_features.head()
# all_audio_features.tail()

In [ ]:
# STEP 2
# df_billboard, append
    # df_billboard_songs
# set missing audio = np.nan
    # create dataframe of billboard songs not on spotify
    # create dataframe of billboard songs not on spotify
# sort, reset index




### Merge All Songs With Metadata

In [ ]:
# TODO
# df_8M_songs, append
    # df_1M_songs
    # df_billboard_songs
    # remove duplicates, sort, reset index
    
    

In [ ]:
# separate songs into
    # not on spotify (no id)
    # matched with audio features
    
    
 

In [ ]:
# check the percentage of songs accounted for




In [ ]:
# now merge with entire billboard list



# re-count the percentage of songs accounted for in the entire set

### Merge all Datasets into Billboard Features, 
export Billboard Features and All Features

In [ ]:
# QA check vs QA dataframe


